___
# Insper - Ciência dos Dados - Projeto 1

# Obtenção das notícias

Utilize este notebook para **baixar notícias** sobre algum assunto de seu interesse.

As notícias serão automaticamente baixadas do site https://www.istoedinheiro.com.br/ utilizando a **seção** de notícias e **quantidade de páginas**

Será gerado um arquivo `dados.xlsx` contendo as informações disponíveis para o projeto.

## Preparação do ambiente no jupyter

Vamos importar algumas bibliotecas e definir algumas funções úteis!

In [10]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import os.path
import time
import random
from math import floor


from ipywidgets import IntProgress
from IPython.display import display

In [2]:
def parse_result(content, page=1, secao=''):
    
    soup = BeautifulSoup(content, 'html.parser')

    lista_tag_noticia = soup.find_all('article', class_='thumb')

    lista_titulo = []
    lista_desc = []
    lista_data = []
    lista_page = []

    for i in range(0, len(lista_tag_noticia)):

        tag_noticia = lista_tag_noticia[i]

        titulo = tag_noticia.find('h3').text
        titulo = titulo.replace('\n', '')
        lista_titulo.append(titulo)

        descricao = tag_noticia.find('p').text
        lista_desc.append(descricao)

        data_hora = tag_noticia.find('time', class_='c-data').text
        lista_data.append(data_hora)

    df = pd.DataFrame({'Categoria': secao,
                       'Titulo': lista_titulo,
                       'Descrição': lista_desc,
                       'Data': lista_data,
                       'Pagina': page,
                       'Target': None
                      })
    return df
    
def get_content(url):
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    resposta = requests.get(url=url, headers=headers, timeout=30)
    resposta.encoding = 'utf-8'
    return resposta.text

def get_news_page(secao, page):
    url = f'https://www.istoedinheiro.com.br/categoria/{secao}/page/{page}/'
    content = get_content(url)
    df_res = parse_result(content, page, secao)
    return df_res

def get_df_news(assuntos_noticias, n_pages=3):  
    print('\nProgresso:')
    max_not = n_pages*len(assuntos_noticias)
    prog_bar = IntProgress(min=0, max=max_not)
    display(prog_bar)

    for page in range(1, n_pages+1):
        for i, assunto in enumerate(assuntos_noticias):
            df_aux = get_news_page(assunto, page)
            
            if page==1 and i==0:
                df = df_aux
            else:
                df = pd.concat([df, df_aux], ignore_index=True)
        
            prog_bar.value += 1
            sec_sleep = random.randint(2, 5)
            time.sleep(sec_sleep)
            
    return df

def get_news(assuntos_noticias, n_pages=3, perc_train=0.6):
       
    if not os.path.isfile('../../data/dados.xlsx'):
        
        print(f'Ok! Vou baixar notícias sobre nas seções {assuntos_noticias} no IstoEDinheiro.com.br!')
        print('\nAguarde... Este processo pode demorar alguns minutos!')
        
        df = get_df_news(assuntos_noticias, n_pages)
        df = df.sample(frac=1.0)
    
        writer = pd.ExcelWriter('../../data/dados.xlsx')
        
        n_real = len(df)
        n_treino = floor(n_real * perc_train)

        dft = df.iloc[:n_treino]
        dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

        dfc = df.iloc[n_treino:]
        dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

        writer.save()
        
        print('Processo finalizado!')
        print('\nGeramos um arquivo chamado dados.xlsx na pasta data, confira!')
        
    else:
        
        raise Exception('Arquivo dados.xlsx já existe na pasta data! Apague o arquivo caso queira gerar novamente!')


## Defina as Categorias

Defina na próxima célula uma lista com as seções ou categorias do site **IstoEDinheiro** nas quais gostaria de encontrar notícias para o Projeto 1.

Precisa ser uma categoria válida no site

Exemplos:
- estilo
- economia
- black-friday
- negocios
- mercado-digital
- financas
- giro

In [3]:
# Quer pesquisar notícias em quais seções do site?!
# Ex: assuntos_noticias = ['estilo', 'economia']
assuntos_noticias = ['estilo']

In [4]:
if len(assuntos_noticias) == 0:
    raise Exception('Defina pelo menos um assunto na lista assuntos_noticias!')

## Defina a quantidade de páginas

Execute a próxima célula e escolha a quantidade de páginas a serem baixadas.

**ATENÇÃO**: cada página contém múltiplas notícias!

In [5]:
# Altere aqui!
# Defina quantas páginas de cada seção quer recuperar.
qtde_paginas = 500

In [6]:
if qtde_paginas <= 0:
    raise Exception('Você precisa definir a quantidade de páginas!')

## Obtenção das notícias

Execute a próxima célula. Ela irá construir uma base de dados de notícias sobre o seu assunto de interesse!

Este processo poderá demorar alguns minutos. Quando ele finalizar, será gerado um arquivo `dados.xlsx` com as notícias!

Pontos de atenção:
- Confira se a quantidade de notícias encontradas foi suficiente (conforme enunciado)

In [8]:
get_news(assuntos_noticias, n_pages=qtde_paginas, perc_train=0.6)

Ok! Vou baixar notícias sobre nas seções ['estilo'] no IstoEDinheiro.com.br!

Aguarde... Este processo pode demorar alguns minutos!

Progresso:


IntProgress(value=0, max=500)

AttributeError: 'NoneType' object has no attribute 'text'